# Example 1 - Default Approach

This notebook demonstrates how to use the TX2 dashboard with a sequence classification transformer using the default approach as described in the "Basic Usage" docs.

In [1]:
%cd -q ..
%load_ext autoreload
%autoreload 2
%matplotlib inline

We enable logging to view the output from `wrapper.prepare()` further down in the notebook. (It's a long running function, and logs which step it's on.)

In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [3]:
import numpy as np
import torch
from tqdm.notebook import tqdm
# import tqdm
from torch.utils.data import Dataset, DataLoader
from torch import cuda
from transformers import AutoModel, AutoTokenizer, BertTokenizer
import pandas as pd

In this example notebook, we use the 20 newsgroups dataset, which can be downloaded through sklearn via below:

In [4]:
from sklearn.datasets import fetch_20newsgroups

train_data = fetch_20newsgroups(subset='train')
test_data = fetch_20newsgroups(subset='test')

device = 'cuda' if cuda.is_available() else 'cpu'

Defined below is a simple sequence classification model with a variable for the language model itself `l1`. Since it is a BERT model, we take the sequence embedding from the `[CLS]` token (via `output_1[0][:, 0, :])`) and pipe that into the linear layer.

In [5]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained("bert-base-cased")
        self.l2 = torch.nn.Linear(768, 20)

    def forward(self, ids, mask):
        output_1= self.l1(ids, mask)
        output = self.l2(output_1[0][:, 0, :]) # use just the [CLS] output embedding
        return output
    
model = BERTClass()
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Some simplistic data cleaning, and putting all data into dataframes for the wrapper

In [6]:
def clean_text(text):
    text = text[text.index("\n\n")+2:]
    text = text.replace("\n", " ")
    text = text.replace("    ", " ")
    text = text.replace("   ", " ")
    text = text.replace("  ", " ")
    text = text.strip()
    return text

In [7]:
train_rows = []
for i in range(len(train_data["data"])):
    row = {}
    row["text"] = clean_text(train_data["data"][i])
    row["target"] = train_data['target'][i]
    if row["text"] == "" or row["text"] == " ": continue
    train_rows.append(row)
train_df = pd.DataFrame(train_rows)

In [8]:
test_rows = []
for i in range(len(test_data["data"])):
    row = {}
    row["text"] = clean_text(test_data["data"][i])
    row["target"] = test_data['target'][i]
    if row["text"] == "" or row["text"] == " ": continue
    test_rows.append(row)
test_df = pd.DataFrame(test_rows)

## Training

This section minimally trains the classification and language model - nothing fancy here, just to give the dashboard demo something to work with. Most of this is similar to the huggingface tutorial notebooks.

In [9]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-05)

In [10]:
class EncodedSet(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        print(self.len)

    def __getitem__(self, index):
        text = str(self.data.text[index])
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.target[index], dtype=torch.long)
        }

    def __len__(self):
        return self.len
    
train_set = EncodedSet(train_df, tokenizer, 256)
test_set = EncodedSet(test_df[:1000], tokenizer, 256)

train_params = {'batch_size': 16,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': 2,
                'shuffle': True,
                'num_workers': 0
                }

# put everything into data loaders
train_loader = DataLoader(train_set, **train_params)
test_loader = DataLoader(test_set, **test_params)

11296
1000


In [11]:
def train(epoch):
    model.train()

    
    loss_history = []
    for _,data in tqdm(enumerate(train_loader, 0), total=len(train_loader), desc=f"Epoch {epoch}"):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask).squeeze()

        optimizer.zero_grad()
        loss = loss_function(outputs, targets)
        if _%100==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        loss_history.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         torch.cuda.empty_cache()
    return loss_history

In [12]:
losses = []
for epoch in range(1):
    losses.extend(train(epoch))

/home/81n/miniconda3/envs/tx2/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2136: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Loss:  3.0206263065338135
Epoch: 0, Loss:  2.1934101581573486
Epoch: 0, Loss:  1.4502356052398682
Epoch: 0, Loss:  1.2169358730316162
Epoch: 0, Loss:  0.9175316095352173
Epoch: 0, Loss:  0.7663290500640869
Epoch: 0, Loss:  0.241717129945755
Epoch: 0, Loss:  0.5925803184509277



The wrapper uses an `encodings` dictionary for various labels/visualizations, and can be set up with something similar to:

In [12]:
encodings = {}
for index, entry in enumerate(train_data["target_names"]):
    encodings[entry] = index
encodings

{'alt.atheism': 0,
 'comp.graphics': 1,
 'comp.os.ms-windows.misc': 2,
 'comp.sys.ibm.pc.hardware': 3,
 'comp.sys.mac.hardware': 4,
 'comp.windows.x': 5,
 'misc.forsale': 6,
 'rec.autos': 7,
 'rec.motorcycles': 8,
 'rec.sport.baseball': 9,
 'rec.sport.hockey': 10,
 'sci.crypt': 11,
 'sci.electronics': 12,
 'sci.med': 13,
 'sci.space': 14,
 'soc.religion.christian': 15,
 'talk.politics.guns': 16,
 'talk.politics.mideast': 17,
 'talk.politics.misc': 18,
 'talk.religion.misc': 19}

## TX2

This section shows how to put everything into the TX2 wrapper to get the dashboard widget displayed.

In [13]:
from tx2.wrapper import Wrapper
from tx2.dashboard import Dashboard

ImportError: Numba needs NumPy 1.20 or less

In [15]:
wrapper = Wrapper(
    train_texts=train_df.text,
    train_labels=train_df.target,
    test_texts=test_df.text[:2000],
    test_labels=test_df.target[:2000],
    encodings=encodings, 
    classifier=model, 
    language_model=model.l1, 
    tokenizer=tokenizer,
    overwrite=True
)
wrapper.prepare()

INFO:root:Cache path not found, creating...
INFO:root:Checking for cached predictions...
INFO:root:Running classifier...
/home/81n/miniconda3/envs/tx2/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2136: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
INFO:root:Saving predictions...
INFO:root:Writing to data/predictions.json
INFO:root:Done!
INFO:root:Checking for cached embeddings...
INFO:root:Embedding training and testing datasets
INFO:root:Saving embeddings...
INFO:root:Writing to data/embedding_training.json
INFO:root:Writing to data/embedding_testing.json
INFO:root:Done!
IN

/home/81n/miniconda3/envs/tx2/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2136: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
INFO:root:Saving salience maps...
INFO:root:Writing to data/salience.pkl.gz


INFO:root:Done!
INFO:root:Checking for cached cluster profiles...
INFO:root:Clustering projections...
INFO:root:Saving cluster profiles...
INFO:root:Writing to data/clusters.json
INFO:root:Writing to data/cluster_profiles.pkl.gz
INFO:root:Done!
/home/81n/miniconda3/envs/tx2/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2136: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/home/81n/miniconda3/envs/tx2/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2136: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, us

In [16]:
%matplotlib agg
import matplotlib.pyplot as plt
dash = Dashboard(wrapper)
dash.render()

/home/81n/miniconda3/envs/tx2/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2136: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


To play with different UMAP and DBSCAN arguments without having to recompute the entire `prepare()` function, you can use `recompute_projections` (which will recompute both the projections and visual clusterings) or `recompute_visual_clusterings` (which will only redo the clustering)

In [17]:
# wrapper.recompute_visual_clusterings("KMeans", clustering_args=dict(n_clusters=18))
# wrapper.recompute_visual_clusterings("OPTICS", clustering_args=dict())
# wrapper.recompute_projections(umap_args=dict(min_dist=.2), dbscan_args=dict())

To test or debug the classification model/see what raw outputs the viusualizations are getting, or create your own visualization tools, you can manually call the `classify()`, `soft_classify()`, `embed()` functions, or get access to any of the cached data as seen in the bottom cell

In [18]:
wrapper.classify(["testing"])

[6]

In [19]:
wrapper.soft_classify(["testing"])

[[-0.8178814649581909,
  -0.1121731549501419,
  -0.5409483313560486,
  0.21581129729747772,
  0.2682485580444336,
  -0.6747360229492188,
  1.249107003211975,
  0.9179522395133972,
  1.154718041419983,
  -0.9459081292152405,
  0.5081962943077087,
  -0.27006587386131287,
  0.19237199425697327,
  -0.34899014234542847,
  0.6782020330429077,
  0.04995761811733246,
  -0.22134341299533844,
  -0.2886699438095093,
  -0.5444299578666687,
  -0.4451519846916199]]

In [20]:
wrapper.embed(["testing"])

[[0.6664102077484131,
  0.37331387400627136,
  0.337298721075058,
  -0.8685522079467773,
  -0.6003721356391907,
  -0.25507545471191406,
  0.12168938666582108,
  0.04850916191935539,
  0.29078394174575806,
  -1.1002589464187622,
  0.1251598745584488,
  0.1908818483352661,
  0.017635632306337357,
  0.39517685770988464,
  -0.3626023530960083,
  0.8305122256278992,
  0.3448742628097534,
  0.6485928893089294,
  0.38122469186782837,
  -0.3653854727745056,
  0.3282410800457001,
  -0.651463508605957,
  0.9279282093048096,
  -0.8284140825271606,
  -0.014791159890592098,
  -0.04969337582588196,
  0.1139974296092987,
  0.08380938321352005,
  -0.05369633436203003,
  0.04894104227423668,
  -0.9453793168067932,
  0.38021811842918396,
  -0.11028444766998291,
  0.21194276213645935,
  -0.03325001150369644,
  -0.30094578862190247,
  0.38758620619773865,
  -0.05283786728978157,
  -0.5122196078300476,
  -0.3037961721420288,
  -0.9389336109161377,
  -0.24832458794116974,
  0.5926793813705444,
  -0.45748156

In [21]:
# cached data:
# wrapper.embeddings_training
# wrapper.embeddings_testing
# wrapper.projector
# wrapper.projections_training
# wrapper.projections_testing
# wrapper.salience_maps
# wrapper.clusters
# wrapper.cluster_profiles
# wrapper.cluster_words_freqs
# wrapper.cluster_class_word_sets